---
layout: post
title: API do IBGE
subtitle: Exercícios e Referências
tags: [python, pycharm, jupyter, package, ibge, api]
image: /img/posts/pandas_icon.png
bigimg: /img/posts/pandas_big.png
gh-repo: michelmetran/api_ibge
gh-badge: [follow, star, watch, fork]
comments: true

---

A <a title="Link da API" href="https://servicodados.ibge.gov.br" target="_blank">**_API do IBGE_**</a> possibilita baixar os dados diretamente para o _script_. Para baixar as malhas, ou seja, informações geoespaciais, é possivel baixar os dados em três diferentes formatos:
- **_?formato=application/vnd.geo+json_**, para baixar os dados em GeoJson;
- **_?formato=application/json_**, para baixar os dados em TopoJson;
- **_?formato=image/svg+xml_**, para baixar os dados em SVG;

É possível definir a resolução que, na prática, refere-se ao nível de detalhamento do dado obtido.
- **_?resolucao=0_**, Nenhuma divisão político-administrativa é inserida no interior da malha
- **_?resolucao=1_**, Inclui na malha as macrorregiões. Válido apenas quando a localidade for BR.
- **_?resolucao=2_**, Inclui na malha as Unidades da Federação. Válido apenas quando a localidade for BR ou uma macroregião
- **_?resolucao=3_**, inclui na malha as mesorregiões. Válido apenas quando a localidade for BR, macroregião ou Unidade da Federação
- **_?resolucao=4_**, Inclui na malha as microrregiões. Válido apenas quando a localidade for BR, macroregião, Unidade da Federação ou mesorregião
- **_?resolucao=5_**, inclui na malha os municípios

E a qualidade.
- **_?qualidade=1_**, pior qualidade;
- **_?qualidade=2_**, razoável qualidade;
- **_?qualidade=3_**, boa qualidade;
- **_?qualidade=4_**, melhor qualidade;


{: .alert .alert-danger}
**Aviso:** Esse _post_ tem a finalidade de mostrar os comandos básicos e me deixar com uma "cola" rápida para meu uso cotidiano. Todas os códigos são exemplificativos e podem/devem ser alterados, indicando o nome dos arquivos e diretórios corretamente.

{: .box-note}
**Nota:** É possível acessar esse _post_ em formato <a title="Link do Folium" href="https://github.com/michelmetran/api_ibge/raw/master/docs/api_ibge.pdf" target="_blank">**_pdf_**</a>, diretamente por meio do <a title="Link do Repositório" href="https://github.com/michelmetran/api_ibge" target="_blank">**repositório do GitHub**</a> ou ainda, de maneira interativa, usando o [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/michelmetran/api_ibge/master).

<br>

# Pastas: início do projeto
Inicialmente faz-se necessário criar uma pasta que receberão os dados do IBGE.

In [ ]:
%run '../codes/files/create_folders.py'
create_folders('')

# _Download_ dos dados geoespaciais
Com a estrutura de pastas criada, é possivel fazer o download dos arquivos disponiblizados pelo IBGE. Há uma infinidade de dados e ainda, há a <a title="Link da API" href="http://api.sidra.ibge.gov.br" target="_blank">**_API do SIDRA_**</a> que possibilita obter mais dados.

In [ ]:
import os
import urllib.request
import shutil

# Define qual o código IBGE do Estado
estado = 35

# Define o nome do arquivo que será salvo as informações do IBGE
filename = os.path.join('data', 'divadmin.json')

url = ('https://servicodados.ibge.gov.br/api/v2/malhas/'+
       str(estado)+'/?'+
       '&formato=application/vnd.geo+json'+
       '&resolucao=5'+
       '&qualidade=4')

# Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
       shutil.copyfileobj(response, out_file)

Enfrentei problemas com o encoding do arquivo baixado. Com a função abaixo é possivel conferir que o encoding está correto (_ascii_) para dar continuidade.

In [ ]:
%run '../codes/files/predict_encoding.py'
file_encoding = predict_encoding(filename)
print(file_encoding)

In [ ]:
import folium

# Cria o mapa
webmap = folium.Map(
    location=[-23.9619271,-46.3427499],    
    zoom_start=1
)

folium.GeoJson(filename, name='Trajetos').add_to(webmap)

# Fit and Plot map
webmap.fit_bounds(webmap.get_bounds())
webmap

Ou ainda, ao invés de baixar o arquivo, é possivel fazer com o que o mapa seja criado com a leitura dos dados diretamente do site do IBGE. Nessa função o encoding já foi definido, evitando o problema mencionado acima.

In [ ]:
import json
import urllib.request

webURL = urllib.request.urlopen(url)
data = webURL.read()
encoding = webURL.info().get_content_charset('utf-8')

JSON_object = json.loads(data.decode(encoding))
#JSON_object

In [ ]:
import folium

# Cria o mapa
webmap = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=8
)

folium.GeoJson(JSON_object, name='Trajetos').add_to(webmap)

# Fit and Plot map
webmap.fit_bounds(webmap.get_bounds())
webmap

Uma vez com o mapa na mão, de qualquer que seja o meio que foi obtido, é possivel analisar a "tabela de atributos".
Lá descobrimos que existe o par de coordenadas que define o centroide e, ainda, o 'codarea' que tem o código do IBGE do município.

# _Download_ dos dados tabulares


In [ ]:
import os
import urllib.request
import shutil

# Define qual o código IBGE do Estado
estado = 35

# Define o nome do arquivo que será salvo as informações do IBGE
filename = os.path.join('data', 'tab.json')

url = ('http://servicodados.ibge.gov.br/api/v1/localidades/estados/'+
       str(estado)+
       '/municipios')

# Faz o download do arquivo da 'url' e salva localmente com o nome do arquivo
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
       shutil.copyfileobj(response, out_file)

# Missão é:

- Renomear o campo.
- Dar joins entre jsons
- plotar colorido!

In [ ]:
import pandas as pd


tab = pd.DataFrame(JSON_object)
tab

In [ ]:
# Seleciona Colunas
tab = tab[['id','nome']]

# Renomeia Colunas
tab = tab.rename(columns=lambda x: x.replace('id', 'id_ibge'))
tab

In [ ]:
import json
import pandas as pd
import geopandas as gpd

In [ ]:
#Change the data type and column name to match the geojson file

gdf = gpd.read_file(os.path.join('data', 'divadmin.json'))
gdf

#merge = gdf.merge(tab2, how='left', on='codarea')

In [ ]:
webmap = folium.Map(
    location=[-23.9619271,-46.3427499],
    zoom_start=8
)

folium.Choropleth(
    geo_data=gdf,
    name='choropleth',
    data=tab,
    columns=['id_ibge', 'nome'],
    key_on='features.codearea',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate %'
).add_to(webmap)

webmap

In [ ]:
def replace_in_json_file(filename, from_str, to_str):
    with open(filename) as f:
        data = f.read()
    with open(filename, 'w') as f:
        f.write(data.replace(from_str, to_str))

In [ ]:
def replace_in_json_file(obj, from_str, to_str):
    for key in obj.keys():
        print(key)
        new_key = key.replace(from_str, to_str)
        if new_key != key:
            obj[new_key] = obj[key]
            del obj[key]
    return obj

In [ ]:
#replace_in_json_file(JSON_object, 'codarea', 'id_ibge')
import json
#json_dict = json.loads(JSON_object)

#for item in json_dict[features]:
#    print(item)
#JSON_object
JSON_object['features']

In [ ]:
# Replace string in keys of a Json object
def replace_string(obj, from_str, to_str):
    for key in obj.keys():
        print(key)
        new_key = key.replace(from_str, to_str)
        if new_key != key:
            obj[new_key] = obj[key]
            del obj[key]
    return obj


replace_string(JSON_object, 'crs', 'crs2')
#new_json = json.loads(data, object_hook=remove_dot_key)

In [ ]:
JSON_object

In [ ]:
#url = 'http://portalgeo.seade.gov.br/wp-content/uploads/2019/03/LimiteMunicipal_IGC.zip'
# Se lê certo, no notepad aparecer UTF-8 BOM WO




import json
import urllib.request

webURL = urllib.request.urlopen(url)
data = webURL.read()
encoding = webURL.info().get_content_charset('utf-8')

JSON_object = json.loads(data.decode(encoding))
JSON_object

# Referências

___
<br>

# Exportando o _Juptyter Notebook_ para outros formatos
O arquivo _.ipynb_ pode ser exportado em formatos diversos. Abaixo carrego uma função que escrevi para facilitar o processo de exportação do arquivo em diferentes locais do PC para, posteriormente, atualizar os repositórios contidos no <a title="Link do GitHub" href="https://github.com/michelmetran" target="_blank">_GitHub_</a>.

In [ ]:
# %load '~/Documents/SourceCode/codes/files/export_jupyter.py'
def export_jupyter(path, extensions=['html', 'markdown', 'latex', 'pdf', 'python'], today=True):
    """
    Export .ipynb file to others formats
    :return: File in other formats
    """
    # Import Packages
    import os
    import datetime

    # Data
    timestamp = datetime.datetime.now()
    srt_today = (str(timestamp.year) + '-' +
                 str(f"{timestamp.month:02d}") + '-' +
                 str(f"{timestamp.day:02d}"))

    # Extensions
    for extension in extensions:
        if today==True:
            os.system('jupyter nbconvert --to {} {} --output {}'.
                      format(extension, get_jupyternotebook_name(),
                             os.path.join(path, srt_today+'-'+get_jupyternotebook_name().split('.')[0])))
            print('Arquivo {} exportado corretamente para o formato {} usando prefixo da data.'.
                  format(get_jupyternotebook_name(), extension))

        else:
            os.system('jupyter nbconvert --to {} {} --output {}'.
                      format(extension, get_jupyternotebook_name(),
                             os.path.join(path, get_jupyternotebook_name().split('.')[0])))
            print('Arquivo {} exportado corretamente para o formato {} sem usar prefixo da data.'.
                  format(get_jupyternotebook_name(), extension))


In [ ]:
%run '~/Documents/SourceCode/codes/files/get_jupyternotebook_name.py'

Com as funções para exportar o _Jupyter Notebook_ e para obter o nome do arquivo _.ipynb_ carregadas, basta exportar o arquivo, inicialmente para a pasta _docs_ dentro do projeto e também, visando atualizar os _posts_ do site, para a respectiva pasta.

In [ ]:
export_jupyter('docs',['pdf'], False)
export_jupyter('/home/michel/Documents/SourceCode/michelmetran.github.io/_posts', ['markdown'], True)

<br>

# Atualizando Repositórios
Após as exportações dos arquivos nos formatos necessários, basta atualizar o repositório diretamente pelo  _Jupyter Notebook_.
Abaixo é atualizado o repositório desse projeto específico, bem como a derivação desse projeto no <a title="Link do Folium" href="https://michelmetran.github.io/" target="_blank">**_site_**</a>.

In [ ]:
%run '../codes/git/update_github.py'

In [ ]:
git_full('.', '.', 'Atualizando')
#git_full('/home/michel/Documents/SourceCode/michelmetran.github.io', '.', 'Atualizando')

# _Requirements_
Abaixo é criado o arquivo _requirements.txt_ na raiz do projeto para possibilitar o correto funcionamento do _Jupyter Notebook_ no <a title="Link do My Binder" href="https://mybinder.org/" target="_blank">**_My Binder_**</a>. Após a criação do arquivo, sugere-se a edição manual, visando manter apenas os _packages_ realmente essenciais, listados com o comando _import_ no início do _script_.

In [ ]:
#pip freeze > requirements.txt